In [3]:


#Printing Options For Figures
np.set_printoptions(threshold=sys.maxsize)

In [4]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import shapefile
import pyproj


#! /usr/bin/env python3

# data libraries
import sys
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from netCDF4 import Dataset,num2date  # http://code.google.com/p/netcdf4-python/
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from matplotlib.path import Path

# plotting libraries

# plotting libraries
import matplotlib.pyplot as plt 
import matplotlib.style as style
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point 
import cartopy.mpl.ticker as cticker
from cartopy.feature import LAND,BORDERS,OCEAN
import glob
import pickle

In [5]:
def getPolyCoords(row, geom, coord_type):
    """
    Returns the coordinates ('x|y') of edges/vertices of a Polygon/others
    Args:
    - row: the row object from a geodataframe; i.e.   df.loc[1]
    - geom: the name of "geometry" column, usually "geometry"
    - coord_type: 'x', or 'y'
    ...
    Valid geometrie types 'Polygon', 'Point', 'LineString', 'MultiLineString', 'MultiPolygon'
    """

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        # updated code, using .geoms
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.xy[1] ))
        return all_xy

    if geometry.type=='MultiPolygon':
        all_xy = []
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.exterior.coords.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.exterior.coords.xy[1] ))
        return all_xy

    else:
        # Finally, return empty list for unknown geometries
        return []

In [ ]:
def plotting_iceshelf(dictionary,n):    
    
    max_list,min_list = min_and_max_finder(dictionary,n)
    
    fig = plt.figure(figsize=(15,12))
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_global()
    ax.coastlines(resolution="110m",linewidth=1)
    ax.gridlines(linestyle='--',color='black', dms=True, x_inline=False, y_inline=False)
    ax.coastlines(resolution="110m",linewidth=1)
    ax.set_extent([-180,180, -90, -65],ccrs.PlateCarree())
    
    vmin=np.min(min_list)
    vmax=np.max(max_list)
    
    clevs = np.arange(np.floor(vmin).astype('int'),np.ceil(vmax).astype('int'),3)
    
    for key in dictionary: 
        
        shelf_df = dictionary[key]
        
        Model_name = shelf_df['Model'][n]
        
        temp = np.zeros((len(shelf_df['cropped temp'][n]),len(shelf_df['cropped temp'][n][0])))
        for i in range(len(shelf_df['cropped temp'][n])):
            temp[i] = shelf_df['cropped temp'][n][i]
            
        
        var = np.mean(temp, axis = 0) #2meter surface temperture
        lat = shelf_df['cropped lat'][n] #latitude
        lon = shelf_df['cropped lon'][n] #longitude

        data_pl = var

        lat_pl = lat
        lon_pl = lon

        plt.tricontourf(lon_pl,lat_pl,data_pl,clevs,transform=ccrs.PlateCarree(),cmap='coolwarm')
        
    cb = plt.colorbar(ax=ax, orientation="vertical")
    cb.set_label(Model_name + r' $\degree$ C',size=12,rotation=90,labelpad=15)  
    
    plt.savefig('/raid01/mafields/tas/MODELS_filtered/ssp585/figures/shelves_plots/seasonal/seasonal_shelves_'+Model_name+'.png')
    plt.close()
    
    return fig

In [ ]:
def plotting_iceshelf(dictionary,n):    
    
    fig = plt.figure(figsize=(15,12)) #creating figure
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_global()
    ax.coastlines(resolution="110m",linewidth=1)
    gl = ax.gridlines(draw_labels=False,linestyle='--',color='black', dms=True, x_inline=False, y_inline=True)
    XTEXT_SIZE = 15
    YTEXT_SIZE = 15
    #gl.xlabel_style = {'size': XTEXT_SIZE, 'color': 'k', 'rotation':45, 'ha':'right'}
    #gl.ylabel_style = {'size':YTEXT_SIZE, 'color': 'k', 'weight': 'normal'}
    ax.coastlines(resolution="110m",linewidth=1)
    ax.set_extent([-180,180, -90, -65],ccrs.PlateCarree())
    #ax.set_boundary(map_circle, transform=ax.transAxes)
    ax.add_feature(LAND)
    #ax.add_feature(BORDERS)
    #ax.add_feature(OCEAN)
    
    '''Creating Colorbar levels'''
    vmin=1940 #colorbar min
    vmax=2150 #colorbar max
    
    clevs = np.arange(np.floor(vmin).astype('int'),np.ceil(vmax).astype('int'),1) #specific levels
    
    
    '''Iterating through the ice shelf data. Finds the specific cropped temperature
    , latitudes, and longitudes for each shelf. This code was for plotting the onset year 
    so I filled the size of the cropped temperature array with the year value and plotted 
    it against the latitude and longitude for each shelf.'''
    for key in dictionary:
        
        index = np.where(onset_x['Shelf'] == key )[0][0]
        
        shelf_name = onset_x['Shelf'][index]
        
        year = onset_x['Onset Time'][index]
        
        shelf_df = dictionary[key]
        
        Model_name = shelf_df['Model'][n]
        
        '''creating a temp place holder array to fill with the onset year value for each shelf.
        Code makes an array of all the cropped temps from each model then takes the average of the values.
        The length of the average is used for filling an array of onset year values. (The averageis not
        necessary if using only one model)'''
        
        temp = np.zeros((len(shelf_df['cropped temp'][n]),len(shelf_df['cropped temp'][n][0])))
        for i in range(len(shelf_df['cropped temp'][n])):
            temp[i] = shelf_df['cropped temp'][n][i] 
            
        var = np.mean(temp, axis = 0) 
        var_2 = np.full(len(var),year) #filling the array with onset year
        
        lat = shelf_df['cropped lat'][n] #latitude
        lon = shelf_df['cropped lon'][n] #longitude

        data_pl = var_2

        lat_pl = lat
        lon_pl = lon

        plt.tricontourf(lon_pl,lat_pl,data_pl,clevs,transform=ccrs.PlateCarree(),cmap='Reds_r') # Plotting onset year contour for each shelf
        #CLS = plt.clabel(CS, inline=1, fontsize=10)
        
    cb = plt.colorbar(ax=ax, orientation="vertical",ticks=[2100,2120,2140]) #Creating Colorbar
    cb.set_label(r'Onset Year',size=20,rotation=270) #Colorbar Label
    cb.ax.set_yticklabels(['2100','Threshold Not Met','No Melt Threshold Given']) #Colorbar ticks
    tick_font_size = 12
    cb.ax.tick_params(labelsize=tick_font_size)
    
    #plt.savefig('/raid01/mafields/tas/MODELS_filtered/ssp585/figures/shelves_plots/seasonal/seasonal_shelves_'+Model_name+'.png')
    plt.close()
    
    return fig
